In [1]:
# 神经网络与数字货币量化交易系列（1）——LSTM预测比特币价格
# https://zhuanlan.zhihu.com/p/73444067

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

定义LSTM模型

In [3]:
LSTM = nn.LSTM(input_size=5, hidden_size=10, num_layers=2, batch_first=True)

准备输入的数据

In [4]:
x = torch.randn(3,4,5)
print(x)


tensor([[[-0.9702,  0.2976, -1.5511,  0.3423, -1.1785],
         [ 1.6150, -1.1762, -1.2349, -0.3541, -0.0427],
         [-0.1790, -0.6038, -0.6030,  0.0065,  0.2177],
         [ 0.3728,  0.3140,  1.4856,  0.4893, -0.7760]],

        [[ 0.0199, -0.0327, -0.6525,  1.7078,  0.6704],
         [-1.1280,  0.3733,  2.2598,  1.3574,  0.0945],
         [ 1.3285,  1.4419,  0.8628, -0.3219, -0.3883],
         [ 1.0088,  0.5116,  2.5808,  0.7327, -0.3754]],

        [[-0.0716,  1.4311,  1.4345, -1.9375, -0.0256],
         [ 0.3801, -0.5521, -0.9732,  0.1145, -0.8313],
         [ 1.3806,  1.6487,  0.1712,  0.0044, -0.2758],
         [-1.3982, -2.2070, -0.3427, -1.9353,  0.8173]]])


x的形状为(3,4,5),由于我们之前定义了batch_first=True, 此时的batch_size的大小为3, sqe_len为4, input_size为5。 x[0]代表了第一个batch。

如果没定义batch_first,默认为False,则此时数据的代表的完全不同,batch大小为4, sqe_len为3, input_size为5。 此时x[0]代表了所有batch在t=0时数据,依次类推。 个人感觉这种设定不符合直觉, 所以添加了参数batch_first=True.

两者之间数据的转换也很方便: x.permute(1,0,2)x的形状为(3,4,5),由于我们之前定义了batch_first=True, 此时的batch_size的大小为3, sqe_len为4, input_size为5。 x[0]代表了第一个batch。

如果没定义batch_first,默认为False,则此时数据的代表的完全不同,batch大小为4, sqe_len为3, input_size为5。 此时x[0]代表了所有batch在t=0时数据,依次类推。 个人感觉这种设定不符合直觉, 所以添加了参数batch_first=True.

两者之间数据的转换也很方便: x.permute(1,0,2)

输入和输出

LSTM的输入输出的形状很容易让人迷惑,借助下图可以辅助理解:

img:v2-300d2df24378efd0783909a4687eb4df_720w.jpg

In [5]:
x = torch.randn(3,4,5)
h0 = torch.randn(2, 3, 10)
c0 = torch.randn(2, 3, 10)
output, (hn, cn) = LSTM(x, (h0, c0))
print(output.size()) #在这里思考一下,如果batch_first=False输出的大小会是多少?
print(hn.size())
print(cn.size())

torch.Size([3, 4, 10])
torch.Size([2, 3, 10])
torch.Size([2, 3, 10])


观察输出的结果,和前面的参数的解释一致．注意到hn.size()的第二个值为3,和batch_size的大小保持一致,说明hn中并没有保存中间状态,只保存了最后一步。
由于我们的LSTM网络有两层,其实hn最后一层的输出就是output的值,output的形状为[3, 4, 10],保存了t=0,1,2,3所有时刻的结果,所以:

In [6]:
hn[-1][0] == output[0][-1] #第一个batch在hn最后一层的输出等于第一个batch在t=3时output的结果
hn[-1][1] == output[1][-1]
hn[-1][2] == output[2][-1]

tensor([True, True, True, True, True, True, True, True, True, True])

5.准备比特币行情数据
前面讲了这么多内容,只是铺垫,理解LSTM的输入输出很是重要,否则随意从网上摘抄一些代码很容易出错,由于LSTM在时间序列上的强大能力,即使模型是错误的,最后也能得出的不错结果。

数据的获取

数据使用的是Bitfinex交易所BTC_USD交易对的行情数据。

In [13]:
import requests
import json
resp = requests.get('https://www.quantinfo.com/API/m/chart/history?symbol=BTC_USD_BITFINEX&resolution=60&from=1525622626&to=1562658565')
# print(resp)
data = resp.json()
df = pd.DataFrame(data,columns = ['t','o','h','l','c','v'])
print(df.head(5))

ValueError: check_hostname requires server_hostname